In [2]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import xmltodict
import konlpy
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import re
from collections import Counter
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 과제 0726_2
네이버 카테고리별 기사를 아래 카테고리 정의를 기준으로 크롤링한 후 불필요한 공백을 제거하고 한글만으로 된 데이터 프레임을 출력하세요.
(전체, 카테고리별로키워드 분석을 위한 시각화)
- 카테고리 정의
    - 100 정치
        - (청와대[264] + 국회/정당[265] + 북한[268] + 국방/외교[267])

    - 101 경제
        - (금융[259] + 증권[258] + 산업/재계[261] + 글로벌 경제[262] + 부동산[260])

    - 103 생활/문화
        - (건강정보[241] + 여행/레저[237] + 공연/전시[242] + 날씨[248] + 생활문화 일반[245])

    - 105 IT/과학
        - (통신/뉴미디어[227] + IT 일반[230] + 컴퓨터[283] + 과학 일반[228])
        
- 카테고리별 1000개, 총 4천개로 구성된 데이터프레임 생성

## 크롤링

- https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=100&sid2=265
- sid1, sid2

- https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=265&sid1=100&date=20220725&page=6

In [16]:
cats = {100 : [264, 265, 268, 267], 
        101 : [259, 258, 261, 262, 260], 
        103 : [241, 237, 242, 248, 245], 
        105 : [227, 230, 283, 228] }
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

#===== 실행했을때 DataFrame의 list를 return 합니다.=====#
def get_news():
    dt  = datetime.now()
    dt = dt.strftime('_%m%d_%H%M')
    ind = 0
    list_df = []
    news_df = pd.DataFrame(columns=['category', 'news'])
    with open('./logs/log'+dt+'.txt', 'w') as fw:
        # ===== 대분류 =====#
        for cat, subcats in cats.items():
            fw.write('*'*30+'\n')
            fw.write('***** start : cat : ' + str(cat)+ '*****\n\n')
            fw.write('*'*30+'\n')

            news_df = pd.DataFrame(columns=['category', 'news'])
            # ===== 소분류 =====#
            for subcat in subcats:
                fw.write('***** start : subcat : ' + str(subcat)+ '*****\n\n')
                
                # ===== 페이지 15개에 대해 크롤링 =====#
                for i in range(15):
                    fw.write('*****start : i : ' + str(i) + '*****\n')

                    url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2='+str(subcat)+'&sid1='+str(cat)+'&date=20220725&page='+ str(i+1)

                    res1 = requests.get(url, headers=headers) 
                    time.sleep(0.3)
                    fw.write('***** url : '+url+'\n')
                    fw.write('***** status_code : ' + str(res1.status_code)+'\n')

                    # ===== 얻은 html에서 a 태그를 추출 =====#
                    bs = BeautifulSoup(res1.text, 'html.parser')
                    links = bs.select('#main_content > div > ul > li > dl > dt > a')
                    fw.write('***** links in url : '+str(len(links))+'\n')

                    # ===== 추출한 a 태그에서 href속성 값 (link) 추출 =====#
                    newslinks = []
                    for j, a in enumerate(links):
                        newslink = a.get('href')
                        newslinks.append(newslink)                
                        fw.write('***** link : '+str(newslink)+'\n')
                    newslinks = list(set(newslinks))

                    # ===== 각 link에서 한국어 추출 ===== #
                    for link in newslinks:
                        res2 = requests.get(link, headers=headers) 
                        time.sleep(0.3)
                        fw.write('***** link : '+link+'\n')
                        fw.write('***** status_code : ' + str(res2.status_code)+'\n')

                        bs2 = BeautifulSoup(res2.text, 'html.parser')
                        news = bs2.find('div', id='dic_area')
                        if news:
                            news = news.text
                            news = re.sub('[^가-힣]+', ' ', news)
                            news = re.sub('\s+', ' ', news)

                            news_df.loc[ind] = [subcat, news]
                            fw.write('***** ind : '+str(ind)+'\tcat : '+str(subcat)+'\tnews : '+news+'\n')
                            ind +=1
                        
                    fw.write('***** end : i : ' + str(i) + '*****\n\n')
                fw.write('***** end : subcat : ' + str(subcat)+ '*****\n\n\n\n')
            fw.write('***** end : cat : ' + str(cat)+ '*****\n\n\n\n')
            list_df.append(news_df)
    return list_df

In [17]:
li_df = get_news()
len(li_df)

4

In [ ]:
#===== pkl 파일로 저장 =====#
# li_df[0].to_pickle('news_100.pkl')
# li_df[1].to_pickle('news_101.pkl')
# li_df[2].to_pickle('news_103.pkl')
# li_df[3].to_pickle('news_105.pkl')

## 저장한 파일 확인

In [19]:
news_100 = pd.read_pickle('news_100.pkl')
news_101 = pd.read_pickle('news_101.pkl')
news_103 = pd.read_pickle('news_103.pkl')
news_105 = pd.read_pickle('news_105.pkl')

In [21]:
news_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088 entries, 0 to 1087
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1088 non-null   int64 
 1   news      1088 non-null   object
dtypes: int64(1), object(1)
memory usage: 25.5+ KB


In [22]:
news_101.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 1088 to 2425
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1338 non-null   int64 
 1   news      1338 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [23]:
news_103.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180 entries, 2426 to 3605
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1180 non-null   int64 
 1   news      1180 non-null   object
dtypes: int64(1), object(1)
memory usage: 27.7+ KB


In [24]:
news_105.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 851 entries, 3606 to 4456
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  851 non-null    int64 
 1   news      851 non-null    object
dtypes: int64(1), object(1)
memory usage: 19.9+ KB


In [5]:
news_100.head()

,category,news
0,264,앵커 윤석열 대통령이 오늘 일 김현숙 여성가족부 장관에게 대선공약인 여가부 폐지 ...
1,264,경찰청장 추천권 무력화 지적엔 추천 제청은 다르다 대통령 지지율 고공행진 발언 더...
2,264,김현숙 장관 단독 업무보고윤석열 대통령이 일 김현숙 여성가족부 장관에게 여가부 업...
3,264,서울 연합뉴스 안정원 기자 윤석열 대통령이 점심식사를 위해 일 서울 중구 한 식당...
4,264,존폐 여부 다시 정치 쟁점화 김현숙 여성가족부 장관이 일 정부서울청사 브리핑실에서...


In [6]:
news_101.head()

,category,news
1088,259,기존판례는 압류걸린 통장에 압류 허용대법원 압류액 초과한 압류는 부당 착오송금 관...
1089,259,카카오뱅크 가 출범 주년을 맞아 고객에 감사하는 의미로 모두의 서프라이즈 이벤트를...
1090,259,이준수 함용일 금감원 부원장보 부원장 승진부원장 명 중 명 교체 김은경 소보처장 ...
1091,259,안심전환대출 대상은 억 이하 주택 대상 월부터 전환 억 만원 한도 최대 올 하반기...
1092,259,대우조선 사태 후폭풍 커지는 산은 책임론 시장의 흐름보다 비용에 방점 둬하청 재하...


In [7]:
news_103.head()

,category,news
2426,241,생활습관을 관리하면 유전적 요인으로 인한 뇌졸중 위험을 일부 상쇄할 수 있다는 연...
2427,241,게티이미지뱅크다한증은 손 겨드랑이 발 등에서 땀이 너무 많이 흐르는 질환이다 한 ...
2428,241,사진 아이클릭아트 한여름에도 손발이 차서 냉장고 문을 열기 꺼려진다면 레이노증후군...
2429,241,연합뉴스코로나 유행의 재확산 국면이 지속되는 가운데 일 오후 시까지 전국에서 만명...
2430,241,눈에 생긴 검은 점은 결막모반 으로 대부분 눈 건강에 영향을 주지 않는다 사진 게...


In [8]:
news_105.head()

,category,news
3606,227,이슈진단 업계는 국회 계류된 법안에 주목 브로드밴드와 넷플릭스가 망 이용계약을 두...
3607,227,류긍선 카카오모빌리티 대표 사회적 책임 고민할 시간 달라 카카오 존중한다 받아들여...
3608,227,명함 앱 리멤버 신입 인턴 채용 플랫폼 슈퍼루키 인수 드라마앤컴퍼니 제공 재판매 ...
3609,227,사 합산 영업익 조 억가입자 증가 마케팅비도 감소중간요금제 도입땐 실적 하향하반기...
3610,227,이스트시큐리티 보안 환경 점검 설문조사 결과 발표 이스트시큐리티 제공 보안 전문 ...
